# Visual Anomaly detection

Problem Statement:

Specific Libs used: Pytorch, Anomlib(Based on Ganomly)

Dataset Used: Mvtech AD( Trsisostor class)

In [1]:
#setting work directory
import os

work_dir = '/transitor/transitor'


In [2]:
#import neccessary libs

import numpy as np
import matplotlib as plt
import yaml, warnings,math,glob, cv2, random
import logging
import pprint

In [3]:
# defining the warning settings
warnings.filterwarnings('ignore')

#setting logger settings for debugging
logger = logging.getLogger("anomalib")

import anomalib
from pytorch_lightning import Trainer, seed_everything
from anomalib.config import get_configurable_parameters
from anomalib.data import get_datamodule
from anomalib.models import get_model
from anomalib.utils.callbacks import LoadModelCallback, get_callbacks
from anomalib.utils.loggers import configure_logger, get_experiment_logger

In [4]:
import torch
print(torch.__version__)
print(torch.version.cuda)
print(torch.backends.cudnn.version())
print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.current_device())
print(torch.cuda.device(0))
print(torch.cuda.get_device_name(0))

1.13.1
11.6
8302
True
1
0
NVIDIA GeForce RTX 3060 Laptop GPU


# Model Config Path

Currently, there are **12** anomaly detection models available in `anomalib` library. Namely, 

- [CFA](https://arxiv.org/abs/2206.04325)
- [Patchcore](https://arxiv.org/pdf/2106.08265.pdf)
- [Padim](https://arxiv.org/pdf/2011.08785.pdf)
- [DFKDE](https://github.com/openvinotoolkit/anomalib/tree/development/anomalib/models/dfkde)
- [DFM](https://arxiv.org/pdf/1909.11786.pdf)
- [CFlow](https://arxiv.org/pdf/2107.12571v1.pdf)
- [Ganomaly](https://arxiv.org/abs/1805.06725)
- [STFPM](https://arxiv.org/pdf/2103.04257.pdf)
- [FastFlow](https://arxiv.org/abs/2111.07677)
- [DREAM](https://arxiv.org/pdf/2108.07610v2.pdf)
- [Reverse Distillation](https://arxiv.org/pdf/2201.10703v2.pdf)
- [EfficientAD](https://arxiv.org/pdf/2303.14535.pdf)




Now, let's get their config paths from the respected folders.

In [5]:
CONFIG_PATHS = 'C:/Users/skrma/anaconda3/envs/Pytorch/Lib/site-packages/anomalib/models'

MODEL_CONFIG_PAIRS = {
    'cfa'      :   f'{CONFIG_PATHS}/cfa/config.yaml',
    'efficientad': f'{CONFIG_PATHS}/efficientad/config.yaml',
    'patchcore':   f'{CONFIG_PATHS}/patchcore/config.yaml',
    'padim':       f'{CONFIG_PATHS}/padim/config.yaml',
    'cflow':       f'{CONFIG_PATHS}/cflow/config.yaml',
    'dfkde':       f'{CONFIG_PATHS}/dfkde/config.yaml',
    'dfm':         f'{CONFIG_PATHS}/dfm/config.yaml',
    'ganomaly':    f'{CONFIG_PATHS}/ganomaly/config.yaml',
    'stfpm':       f'{CONFIG_PATHS}/stfpm/config.yaml',
    'fastflow':    f'{CONFIG_PATHS}/fastflow/config.yaml',
    'draem':       f'{CONFIG_PATHS}/draem/config.yaml',
    'reverse_distillation': f'{CONFIG_PATHS}/reverse_distillation/config.yaml',
}

In [6]:
MODEL = 'efficientad' 

print(open(os.path.join(MODEL_CONFIG_PAIRS[MODEL]), 'r').read())

dataset:
  name: mvtec
  format: mvtec
  path: ./datasets/MVTec
  category: bottle
  task: segmentation
  train_batch_size: 1
  eval_batch_size: 16
  num_workers: 8
  image_size: 256 # dimensions to which images are resized (mandatory)
  center_crop: null # dimensions to which images are center-cropped after resizing (optional)
  normalization: none # data distribution to which the images will be normalized: [none, imagenet]
  transform_config:
    train: null
    eval: null
  test_split_mode: from_dir # options: [from_dir, synthetic]
  test_split_ratio: 0.2 # fraction of train images held out testing (usage depends on test_split_mode)
  val_split_mode: same_as_test # options: [same_as_test, from_test, synthetic]
  val_split_ratio: 0.5 # fraction of train/test images held out for validation (usage depends on val_split_mode)

model:
  name: efficientad
  teacher_out_channels: 384
  model_size: medium # options: [small, medium]
  lr: 0.0001
  weight_decay: 0.00001
  padding: true
  # gen

In [7]:
## Update config

new_update = {
    "path" : '/transitor/transitor',
    "category" : 'transistor',
    "image_size" : 256,
    "train_batch_size": 48,
    "seed" : 101
}

In [8]:
# update yaml key's value

def update_yaml (old_yaml, new_yaml, new_update):
    # load yaml
    with open (old_yaml) as f:
        old = yaml.safe_load(f)
        
    temp = []
    def set_state(old, key, value):
        if isinstance(old, dict):
            for k, v in old.items():
                if k == 'project':
                    temp.append(k)
                if k == key:
                    if temp and k == 'path':
                        # right now not altering 'project.path' configuration
                        continue
                    old[k] = value
                elif isinstance(v, dict):
                    set_state(v, key, value)
                    
    # iterate over the new update key-value pair
    for key, value in new_update.items():
        set_state(old, key, value)
        
    #save the updated/ modified yaml file
    with open(new_yaml, 'w') as f:
        yaml.safe_dump(old, f, default_flow_style = False)

In [9]:
# setting a new path locartion of new config file
new_yaml_path = CONFIG_PATHS + '/' + list(MODEL_CONFIG_PAIRS.keys())[0] + '_new.yaml'

# run the update yaml method to update desired key's values
update_yaml(MODEL_CONFIG_PAIRS[MODEL], new_yaml_path, new_update)

In [10]:
with open(new_yaml_path) as f:
    updated_config = yaml.safe_load(f)
pprint.pprint(updated_config) # check if upadted

{'dataset': {'category': 'transistor',
             'center_crop': None,
             'eval_batch_size': 16,
             'format': 'mvtec',
             'image_size': 256,
             'name': 'mvtec',
             'normalization': 'none',
             'num_workers': 8,
             'path': '/transitor/transitor',
             'task': 'segmentation',
             'test_split_mode': 'from_dir',
             'test_split_ratio': 0.2,
             'train_batch_size': 48,
             'transform_config': {'eval': None, 'train': None},
             'val_split_mode': 'same_as_test',
             'val_split_ratio': 0.5},
 'logging': {'log_graph': False, 'logger': []},
 'metrics': {'image': ['F1Score', 'AUROC'],
             'pixel': ['F1Score', 'AUROC'],
             'threshold': {'manual_image': None,
                           'manual_pixel': None,
                           'method': 'adaptive'}},
 'model': {'lr': 0.0001,
           'model_size': 'medium',
           'name': 'efficientad',

### Prepare Model, Dataloader, Callback with config


In [11]:
if updated_config['project']['seed'] != 0:
    print(updated_config['project']['seed'])
    seed_everything(updated_config['project']['seed'])

Global seed set to 101


101


In [12]:
# It will return the configurable parameters in DictConfig object.
config = get_configurable_parameters(
    model_name = updated_config['model']['name'],
    config_path = new_yaml_path
)

In [13]:
# pass the config file to model, logger , callbacks and datamodule
model = get_model(config)
experiment_logger = get_experiment_logger(config)
callbacks = get_callbacks(config)
datamodule = get_datamodule(config)

In [14]:
# start training
torch.cuda.empty_cache()
trainer = Trainer(**config.trainer, logger=experiment_logger, callbacks=callbacks)
trainer.fit(model=model, datamodule=datamodule)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
`Trainer(limit_train_batches=1.0)` was configured so 100% of the batches per epoch will be used..
`Trainer(limit_val_batches=1.0)` was configured so 100% of the batches will be used..
`Trainer(limit_test_batches=1.0)` was configured so 100% of the batches will be used..
`Trainer(limit_predict_batches=1.0)` was configured so 100% of the batches will be used..
`Trainer(val_check_interval=1.0)` was configured so validation will run at the end of the training epoch..
You are using a CUDA device ('NVIDIA GeForce RTX 3060 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL

Training: 0it [00:00, ?it/s]

Calculate teacher channel mean:   0%|                                                            | 0/5 [00:30<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 3.07 GiB (GPU 0; 6.00 GiB total capacity; 3.22 GiB already allocated; 503.00 MiB free; 3.22 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF